# Elixir: Enumerables & Streams
* The [Enum module](https://hexdocs.pm/elixir/Enum.html) provides a huge range of functions to transform, sort, group, filter and retrieve items from enumerables. It is one of the modules developers use frequently in their Elixir code.

In [1]:
# Elixir provides ranges:
Enum.map(1..3, fn x -> x*2 end)

[2, 4, 6]

In [2]:
Enum.reduce(1..3, 0, &+/2)

6

* For operations like inserting and updating particular elements, you may need modules specific to the data type. For example, if you want to insert an element at a given position in a list, you should use the List.insert_at/3 function from the [List module](https://hexdocs.pm/elixir/List.html), as it would make little sense to insert a value into, for example, a range.

* Enum functions are __polymorphic__ - they can work with diverse data types.For now we are going to move on to a specific kind of enumerable called a stream.

### Eager vs Lazy evaluation
* All Enum functions are eager. Most expect an enumerable and return a list.

In [3]:
odd? = &(rem(&1, 2) != 0)

Enum.filter(1..3, odd?)

[1, 3]

* When performing multiple operations with Enum, each operation is going to generate an intermediate list until we reach the result.

In [4]:
total_sum = 1..100_000 |> Enum.map(&(&1 * 3)) |> 

Enum.filter(odd?) |> Enum.sum

7500000000

* The example above is a __pipelined operation__. We multiply each element in the range by 3. This first operation creates & returns a list with 100_000 items. Then we keep all odd elements from the list, generating a new list, now with 50_000 items, and then we sum all entries.

### [Pipe operator](https://hexdocs.pm/elixir/Kernel.html#%7C%3E/2)
* The __|> symbol__ is the pipe operator. It passes the left-side result as the first argument to the function call on its right side, similar to the Unix | operator. Its purpose is to highlight the data being transformed by a series of functions. 

* To see how it can make the code cleaner, look at the example above rewritten without using the |> operator.

In [5]:
Enum.sum(Enum.filter(Enum.map(1..100_000, &(&1 * 3)), odd?))

7500000000

### [Streams](https://hexdocs.pm/elixir/Stream.html)
* Elixir provides the Stream module which supports lazy operations. Streams are lazy, composable enumerables.

In [7]:
# 1..100_000 |> Stream.map(&(&1 * 3)) returns a stream
# that represents the map computation over the range 1..100_000:

1..100_000 |> Stream.map(&(&1 * 3)) |> Stream.filter(odd?) |> Enum.sum

7500000000

In [8]:
1..100_000 |> Stream.map(&(&1 * 3))

#Stream<[enum: 1..100000, funs: [#Function<48.2459763/1 in Stream.map/2>]]>

In [9]:
1..100_000 |> Stream.map(&(&1 * 3)) |> Stream.filter(odd?)

#Stream<[enum: 1..100000, funs: [#Function<48.2459763/1 in Stream.map/2>, #Function<40.2459763/1 in Stream.filter/2>]]>

* Streams build a series of computations that are invoked only when we pass the underlying stream to the Enum module (instead of building interim lists). Streams are useful when working with large, possibly infinite, collections.
* Many Stream functions accept any enumerable as an argument and return a stream as a result. The module also provides functions for creating streams. For example, Stream.cycle/1 can create a stream that cycles a given enumerable infinitely. Be careful to not call a function like Enum.map/2 on such streams, as they would cycle forever!

In [10]:
stream = Stream.cycle([1, 2, 3])

#Function<64.2459763/2 in Stream.unfold/2>

In [11]:
Enum.take(stream, 10)

[1, 2, 3, 1, 2, 3, 1, 2, 3, 1]

* Stream.unfold/2 can be used to generate values from a given initial value.

In [12]:
stream = Stream.unfold("hełło", &String.next_codepoint/1)

#Function<64.2459763/2 in Stream.unfold/2>

In [13]:
Enum.take(stream, 3)

["h", "e", "ł"]

* Stream.resource/3 can be used to wrap around resources, guaranteeing they are opened right before enumeration and closed afterwards, even in the case of failures. For example, we can use it to stream a file.

In [14]:
stream = File.stream!("some-text-file.txt")

%File.Stream{line_or_bytes: :line, modes: [:raw, :read_ahead, :binary], path: "some-text-file.txt", raw: true}

In [15]:
# fetches first 10 lines of specified file
#Enum.take(stream, 10)

nil

* The Enum and Stream modules can be daunting at first, but you will get familiar with them case by case. Focus on the Enum module first, then move to Stream for scenarios where laziness is required, to either deal with slow resources or large, possibly infinite, collections.